In [1]:
import nbformat
from nbconvert import PythonExporter
import importlib.util
import csv
import json
import os
from datetime import datetime

In [2]:
file_path = "/home/vmuser/Assignments/problem.ipynb"
print("File exists:", os.path.exists(file_path))

File exists: True


In [3]:
!ls

AIChatbot    Pictures	     evaluation
Assignments  Public	     google-chrome-stable_current_amd64.deb
Desktop      Templates	     problem.ipynb
Documents    Videos	     task1
Downloads    auto	     testprojects
Music	     evaluate.ipynb  thinclient_drives


In [4]:
# File paths
problem_file = "/home/vmuser/Assignments/problem.ipynb"
solution_file = "/home/vmuser/evaluation/solution.ipynb"
result_csv = "/home/vmuser/evaluation/evaluation_results.csv"
result_json = "/home/vmuser/evaluation/evaluation_results.json"

In [5]:
# Task weightage (updated as per request)
task_weightage = {
    "load_the_dataset": 2,
    "process_store_data": 1,
    "find_unique_values": 1,
    "total_sales": 1,
    "check_missing_values": 1,
    "sales_distribution": 1,
    "top_customer_segment": 1,
    "regional_purchasing_behavior": 1,
    "high_spending_regions": 1,
    "popular_product_categories": 1,
    "avg_quantity_per_transaction": 1,
    "quantity_sales_relationship": 1,
    "category_quantity_sales_trends": 1,
    "highest_profit_segments": 1,
    "clean_and_calculate_shipping_time": 1,
    "calculate_discounted_price": 1,
    "calculate_revenue_per_day": 1,
    "identify_most_discounted_products": 2,
    "analyze_revenue_efficiency": 1,
    "plot_sales_distribution": 1,
    "plot_sales_by_segmen": 1,
    "plot_sales_trend": 1,
    "plot_order_count_trend": 1,
    "anova_sales_by_category": 2,
    "ttest_sales_by_segment": 2,
    "chi_square_category_region": 2,
    "treat_outliers_iqr": 1,
    "apply_outlier_treatment": 1,
    "compute_correlations": 1,
    "drop_var1": 1,
    "normalize_numeric_columns": 1,
    "one_hot_encode": 2
}

In [6]:
# Generate attempt ID based on timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
submission_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

In [7]:
def load_notebook(notebook_path):
    """Load a Jupyter Notebook and execute it as Python code."""
    with open(notebook_path, 'r', encoding='utf-8') as f:
        nb = nbformat.read(f, as_version=4)
    
    exporter = PythonExporter()
    source_code, _ = exporter.from_notebook_node(nb)
    
    global_namespace = {}
    exec("import pandas as pd\n" + source_code, global_namespace)  # Ensure pandas is available
    return global_namespace


In [8]:
def evaluate_function(user_globals, solution_globals, function_name):
    """Compare user and reference function outputs."""
    if function_name in user_globals and function_name in solution_globals:
        user_func = user_globals[function_name]
        solution_func = solution_globals[function_name]
        
        # Execute both functions
        user_output = user_func()
        solution_output = solution_func()
        
        return int(user_output == solution_output) * task_weightage.get(function_name, 1)
    return 0

In [9]:
def evaluate_submission():
    """Evaluate user's submission against solution notebook."""
    user_globals = load_notebook(problem_file)
    solution_globals = load_notebook(solution_file)
    
    scores = {}
    total_score = 0
    
    for task in task_weightage.keys():
        score = evaluate_function(user_globals, solution_globals, task)
        scores[task] = score
        total_score += score
    
    return scores, total_score

In [11]:
def save_results(scores, total_score):
    """Save evaluation results to CSV and JSON."""
    attempt_id = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    result_entry = {"AttemptID": attempt_id, "Timestamp": timestamp, "TotalScore": total_score, **scores}
    
    # Append to CSV
    df = pd.DataFrame([result_entry])
    if os.path.exists(result_csv):
        df.to_csv(result_csv, mode='a', header=False, index=False)
    else:
        df.to_csv(result_csv, mode='w', header=True, index=False)
    
    # Append to JSON
    if os.path.exists(result_json):
        with open(result_json, "r") as f:
            results = json.load(f)
    else:
        results = []
    results.append(result_entry)
    with open(result_json, "w") as f:
        json.dump(results, f, indent=4)

In [12]:
def main():
    scores, total_score = evaluate_submission()
    save_results(scores, total_score)
    print("Evaluation Completed. Total Score:", total_score)
    print("Detailed Scores:", scores)

if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'pandas'